In [5]:
import requests
import pandas as pd
import os
import dbnomics
from datetime import datetime

# Define the date range
start_date = '2000-01-01'
end_date = '2024-01-01'

# Directory to save the CSV files
data_dir = 'macro_data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [6]:
def fetch_fred_data(series_id, api_key):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json&observation_start={start_date}&observation_end={end_date}"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()['observations']
    df = pd.DataFrame(data)
    df['value'] = df['value'].astype(float)
    return df[['date', 'value']]

FRED_API_KEY = 'e7904868da819fe185bdc305602c0e80'

fred_indicators = {
    'Global Inflation': 'CPIAUCSL',
    'U.S. Unemployment': 'UNRATE',
    'U.S. Interest Rate': 'FEDFUNDS',
    'EU Unemployment': 'LRHUTTTTEUQ156S', 
    'EU Interest Rate': 'IRSTCI01EZM156N'  
}

macro_data_fred = {}
for name, series_id in fred_indicators.items():
    try:
        macro_data_fred[name] = fetch_fred_data(series_id, FRED_API_KEY)
    except requests.HTTPError as e:
        print(f"Failed to fetch data for {name}: {e}")


In [7]:
def check_series_availability(provider_code, dataset_code, series_code):
    url = f"https://api.db.nomics.world/v22/series/{provider_code}/{dataset_code}/{series_code}?observations=1"
    response = requests.get(url)
    return response.status_code == 200

def fetch_dbnomics_data(provider_code, dataset_code, series_code, save_path):
    if os.path.exists(save_path):
        print(f"Data for {series_code} already exists. Skipping download.")
        return pd.read_csv(save_path)
    
    df = dbnomics.fetch_series(provider_code, dataset_code, series_code)
    df = df[['period', 'value']]
    df['value'] = df['value'].astype(float)
    df.to_csv(save_path, index=False)
    return df

oil_producing_countries = [
    'IRN', 'IRQ', 'KWT', 'QAT', 'SAU', 'ARE', 'USA', 'RUS', 'CAN', 'CHN', 'MEX', 
    'VEN', 'BRA', 'NGA', 'NLD', 'NOR', 'KAZ', 'AGO', 'DZA', 'COL', 'GAB', 'OMN', 
    'EGY', 'GNQ', 'LBR', 'LBN', 'LCA', 'LBY', 'MYS', 'SYR', 'SDN', 'TKM', 'TTO', 
    'TUN', 'UZB', 'YEM'
]

def construct_political_stability_series(country_code):
    return f'A-PV.EST-{country_code}'

dbnomics_indicators = {
    'Current Account Balance': ('IMF', 'WEOAGG:2024-04', '001.BCA.us_dollars'),
    'Imports of Goods and Services': ('IMF', 'WEOAGG:2024-04', '001.BM.us_dollars'),
    'Exports of Goods and Services': ('IMF', 'WEOAGG:2024-04', '001.BX.us_dollars'),
    'External Debt, Total': ('IMF', 'WEOAGG:2024-04', '001.D.us_dollars'),
    'General Government Revenue': ('IMF', 'WEOAGG:2024-04', '001.GGR_NGDP.us_dollars'),
    'Unemployment Rate': ('IMF', 'WEOAGG:2024-04', '001.LUR.us_dollars'),
    'Inflation, End of Period Consumer Prices': ('IMF', 'WEOAGG:2024-04', '001.PCPIEPCH.us_dollars'),
    'Gross Domestic Product, Current Prices': ('IMF', 'WEOAGG:2024-04', '001.NGDPD.us_dollars'),
    'Gross Domestic Product, Constant Prices': ('IMF', 'WEOAGG:2024-04', '001.NGDP_RPCH.us_dollars'),
    'Commodity Price Index': ('IMF', 'WEOAGG:2024-04', '001.PALLFNFW.us_dollars'),
    'Crude Oil Prices': ('IMF', 'WEOAGG:2024-04', '001.POILAPSP.us_dollars'),
    'Volume of Imports of Goods and Services': ('IMF', 'WEOAGG:2024-04', '001.TM_RPCH.us_dollars'),
    'Volume of Exports of Goods and Services': ('IMF', 'WEOAGG:2024-04', '001.TX_RPCH.us_dollars'),
    'Trade Volume of Goods and Services': ('IMF', 'WEOAGG:2024-04', '001.TRADEPCH.us_dollars'),
}

for country_code in oil_producing_countries:
    series_code = construct_political_stability_series(country_code)
    dbnomics_indicators[f'Political Stability Index {country_code}'] = ('WB', 'WGI', series_code)

macro_data_db = {}
for name, (provider_code, dataset_code, series_code) in dbnomics_indicators.items():
    save_path = os.path.join(data_dir, f'{name}.csv')
    if check_series_availability(provider_code, dataset_code, series_code):
        try:
            print(f"Fetching data for {name} with series code: {series_code}")
            macro_data_db[name] = fetch_dbnomics_data(provider_code, dataset_code, series_code, save_path)
        except requests.HTTPError as e:
            print(f"Failed to fetch data for {name}: {e}")
    else:
        print(f"Series {series_code} not available for {name}")


Fetching data for Current Account Balance with series code: 001.BCA.us_dollars
Fetching data for Imports of Goods and Services with series code: 001.BM.us_dollars
Fetching data for Exports of Goods and Services with series code: 001.BX.us_dollars
Fetching data for External Debt, Total with series code: 001.D.us_dollars
Series 001.GGR_NGDP.us_dollars not available for General Government Revenue
Series 001.LUR.us_dollars not available for Unemployment Rate
Series 001.PCPIEPCH.us_dollars not available for Inflation, End of Period Consumer Prices
Fetching data for Gross Domestic Product, Current Prices with series code: 001.NGDPD.us_dollars
Series 001.NGDP_RPCH.us_dollars not available for Gross Domestic Product, Constant Prices
Series 001.PALLFNFW.us_dollars not available for Commodity Price Index
Fetching data for Crude Oil Prices with series code: 001.POILAPSP.us_dollars
Series 001.TM_RPCH.us_dollars not available for Volume of Imports of Goods and Services
Series 001.TX_RPCH.us_dollars

In [8]:
all_macro_data = {**macro_data_fred, **macro_data_db}

# Save the data to a consistent format
for key, df in all_macro_data.items():
    save_path = os.path.join(data_dir, f'{key}.csv')
    df.to_csv(save_path, index=False)

metadata = {
    "series_id": list(all_macro_data.keys()),
    "description": list(all_macro_data.keys()),  
    "unit": [""] * len(all_macro_data),  
    "n_observations": [df.shape[0] for df in all_macro_data.values()]
}

metadata_df = pd.DataFrame(metadata)
print(metadata_df.to_markdown(index=False))

| series_id                              | description                            | unit   |   n_observations |
|:---------------------------------------|:---------------------------------------|:-------|-----------------:|
| Global Inflation                       | Global Inflation                       |        |              289 |
| U.S. Unemployment                      | U.S. Unemployment                      |        |              289 |
| U.S. Interest Rate                     | U.S. Interest Rate                     |        |              289 |
| EU Unemployment                        | EU Unemployment                        |        |               80 |
| EU Interest Rate                       | EU Interest Rate                       |        |              289 |
| Current Account Balance                | Current Account Balance                |        |               50 |
| Imports of Goods and Services          | Imports of Goods and Services          |        |            

In [10]:
combined_data = pd.concat(all_macro_data.values(), keys=all_macro_data.keys(), names=['Series', 'Index']).reset_index(level='Series')
combined_data_path = os.path.join(data_dir, 'combined_macro_data.csv')
combined_data.to_csv(combined_data_path, index=False)

In [11]:
combined_data

,Series,date,value,period
Index,,,,
0,Global Inflation,2000-01-01,169.300000,NaT
1,Global Inflation,2000-02-01,170.000000,NaT
2,Global Inflation,2000-03-01,171.000000,NaT
3,Global Inflation,2000-04-01,170.900000,NaT
4,Global Inflation,2000-05-01,171.200000,NaT
...,...,...,...,...
19,Political Stability Index YEM,NaN,-2.996031,2018-01-01
20,Political Stability Index YEM,NaN,-2.770874,2019-01-01
21,Political Stability Index YEM,NaN,-2.647646,2020-01-01
